# Robot stuff

In this notebook we can check how to sample the trajectory of the robot, how to get velocity profiles from the trajectory, which can be used to control the robot, etc.  

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

### Trajectory sampled uniformly in time

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import EPSILON, ROBOT_WIDTH, MM
from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_complexity = 5 #complexity of trajectory
dim = 2 # dimension of setup. 
n_positions = 10
# model = 'full_bandlimited'
model = 'polynomial'
# model = 'bandlimited'

trajectory = Trajectory(n_complexity, dim=dim, model=model, full_period=True)
trajectory.set_coeffs(seed=4)
# trajectory.coeffs = np.zeros_like(trajectory.coeffs)
# trajectory.coeffs[0, 1] = 1
# trajectory.coeffs[1, 3] = 1

times_nonunif = trajectory.get_times(n_samples=n_positions)
basis_nonunif = trajectory.get_basis(times=times_nonunif + 1e-10)
sample_points_nonunif = trajectory.get_sampling_points(basis=basis_nonunif)

plt.figure()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
plt.legend()

### Scale the trajectory to a given bounding box

In [ ]:
from matplotlib.patches import Rectangle

box = np.array([2.0, 2.0])
box = trajectory.scale_bounding_box(box, keep_aspect_ratio=False)
fig, ax = plt.subplots(1)
trajectory.plot(basis_nonunif)
p = Rectangle((0, 0), box[0], box[1], linewidth=1, edgecolor='c',facecolor='none')
ax.add_patch(p)
plt.show()

### Trajectory sampled uniformly in distance

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

times_unif, distances, errors = trajectory.get_times_uniform_in_path(
    n_samples=n_positions,
    time_steps=50000,
    plot=True)
print("cumulative error: {}".format(np.linalg.norm(errors)))
basis_unif = trajectory.get_basis(times=times_unif)
sample_points_unif = trajectory.get_sampling_points(basis=basis_unif)

plt.figure()
# environment.plot()
trajectory.plot(basis_nonunif, color='blue', label='non uniform')
trajectory.plot(basis_unif, color='orange', marker='x', label='uniform')
plt.legend()

## Compute the curvature at each point. 

For now there is still a problem. Since the points are not resampled in time, there is not only a radial acceleration (the one that makes the robot turn) but also a tangent acceleration. In plot below, the orange lines showing the curvature radius, are therefore oriented in the direction of the curve. 

For now I am not quite sure if this is a problem, or if we can still try to incorporate this in the controls for the robot. 

In [ ]:
fig, ax = plt.subplots()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
xlims = ax.get_xlim()
ylims = ax.get_ylim()
plt.axis('equal')

radii, unit_tangents, radial_acc = trajectory.get_curvatures(times_nonunif, ax=ax)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
#plt.xlim([-2, 8])
#plt.ylim([-2, 5])

In [ ]:
# TODO(FD) Below needs some refactoring, 
# there is a lot of duplicate code. 

fig, ax = plt.subplots()
trajectory.plot(basis_unif)
plt.axis('equal')

points_left, points_right = trajectory.get_left_and_right_points(times_unif, width=ROBOT_WIDTH, ax=ax)

In [ ]:
left, right, _ = trajectory.get_left_and_right_arcs(min_max_distance=0.1, plot=True)
plt.show()

## Generate trajectories for both wheels 
This function uses milimiters as units!

In [ ]:
import csv

FOLDER = 'controls/'

def generate_control_parameters(trajectory, 
                                fname='', 
                                min_max_distance=50,
                                curvature_decimals=1,
                                plot=True,
                                comand_decimals=2):
    """Generate control files for the robot.
    This function uses milimeters as units of it's imput and output, thats how they should be passed to the robot
    :param fname: where to save parameters and plots. extension is ignored.  
    :param plot: if True, plot and save figure of trajectory.
    :param curvature_decimals, min_max_distance, width: See trajectory.get_left_and_right_arcs for details. 
    :param command_decimals: precision at which to save control parameters.
    """
    left, right, command_times = trajectory.get_left_and_right_arcs(
        curvature_decimals=curvature_decimals,
        min_max_distance=min_max_distance*MM,
        width=ROBOT_WIDTH)
    
    fname_parts = fname.split('.')
    
    fname = fname_parts[0] 
    
    fnamepng = FOLDER + fname + '.png'
    print(fnamepng)
    if plot:
        basis = trajectory.get_basis(times=command_times)

        fig, ax = plt.subplots()
        trajectory.plot(basis, color='orange', label='trajectory')
        
        ax.set_xlabel('x[m]')
        ax.set_ylabel('y[m]')
        xlims = ax.get_xlim()
        ylims = ax.get_ylim()
        plt.axis('equal')

        pos_left, pos_right = trajectory.get_left_and_right_points(times=command_times, ax=ax, width=ROBOT_WIDTH)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.legend()
        
        # plot the beginning points in a different color. 
        basis_begin = trajectory.get_basis(times=command_times[:3])
        coords_begin = trajectory.get_sampling_points(basis=basis_begin)
        print(coords_begin.shape)
        ax.scatter(*coords_begin[:, 0], color='green')
        ax.scatter(*coords_begin[:, 1:2], color='blue')
        
        plt.title(model)
        #plt.title("{} robot command[s]".format(len(command_times)))
        plt.savefig(fnamepng, bbox_inches='tight')
        print('wrote plots as', fnamepng)
    
    left = np.round(left / MM, comand_decimals)
    right = np.round(right / MM, comand_decimals)
    
    fnametxt = FOLDER + fname + '.gcode'
    with open(fnametxt, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        #header = ['i', 'left_arc', 'right_arc']
        #writer.writerow(header)
        #line = ['G91']
        #writer.writerow(line)
        for i, (r, l) in enumerate(zip(left, right)):
            #line = [i, r, l]
            line = ['G91', 'X{:.2f}'.format(r), 'Y{:.2f}'.format(l)]
            writer.writerow(line)
            if False: #i % 10 == 0:
                #line = ['M114', '']
                line = ['G4','P0.01']
                writer.writerow(line)
                line = ['OCTO1']
                writer.writerow(line)
        print('wrote parameters as', fnametxt)

### Chose parameters of the trajectories to save

In [ ]:
trajectories = [
    dict(
        model = 'bandlimited',
        n_complexity = 3,
        seed = 5,
        controls = dict(
            min_max_distance = 100,
            fname = 'bandlimited1',
        )
    ), 
    dict(
        model = 'polynomial',  
        n_complexity = 3,
        seed = 1,
        controls = dict(
            curvature_decimals = 2,
            fname = 'polynomial1'
        )
    ), 
    dict(
        model = 'full_bandlimited',
        n_complexity = 5,
        seed = 3,
        controls = dict(
            fname = 'full_bandlimited1',
            min_max_distance = 200
        )
    ), 
    dict(
        model = 'circle',
        n_complexity = 5,
        seed = 0,
        controls = dict(
            fname = 'circle1',
        )
    ), 
    dict(
        model = 'eight',
        n_complexity = 5,
        seed = 0,
        controls = dict(
            fname = 'eight1.txt',
            min_max_distance = 100
        )
    ), 
]

def generate_trajectory(seed, n_complexity, model, box=np.array([2000.0, 2000.0]), **kwargs):
    '''
    :param seed: random seed to use for generating parameters. 
    :param n_complexity: complexity degree (K)
    :param box: Dimensions of bounding box.  
    :param model: trajectory model to use (circle or eight). If given, previous paramters are ignored.
    
    :param **kwargs: other kwargs are currently ignored. 
    '''
    # This is a hack, see issue #22: https://github.com/duembgen/SamplingTrajectories/issues/22
    if model == 'circle':
        trajectory = Trajectory(n_complexity=3, model='full_bandlimited') 
        trajectory.coeffs = np.zeros_like(trajectory.coeffs)
        trajectory.coeffs[0, 1] = 1
        trajectory.coeffs[1, 2] = 1
    elif model == 'eight':
        trajectory = Trajectory(n_complexity=5, model='full_bandlimited') 
        trajectory.coeffs = np.zeros_like(trajectory.coeffs)
        trajectory.coeffs[0, 1] = 1
        trajectory.coeffs[1, 3] = 1
    else:
        trajectory = Trajectory(n_complexity=n_complexity, model=model) 
        trajectory.set_coeffs(seed=seed, dimension=2)
    trajectory.scale_bounding_box(box*MM)
    return trajectory
    
for traj in trajectories:
    trajectory = generate_trajectory(**traj)
    generate_control_parameters(trajectory, **(traj['controls']))

## linear trajectories

In [ ]:
box = np.array([2000., 2000.])

trajectories = [
    dict(
        model = 'poly1',
    )
]

def generate_linear(model):
    c0 = np.array([0, 0])
    if model == 'poly1':
        c1 = np.array([4, 0])
    if model == 'poly2':
        c1 = np.array([3, 1])
    if model == 'poly3':
        c1 = np.array([2, 2])
    if model == 'poly4':
        c1 = np.array([1, 3])
    if model == 'poly5':
        c1 = np.array([0, 4])
        
    trajectory = Trajectory(model='polynomial', dim=2, n_complexity=2)
    trajectory.set_coeffs(coeffs=np.c_[c0, c1])
    return trajectory

for traj in trajectories:
    trajectory = generate_linear(traj['model']) 
    generate_control_parameters(trajectory, fname=traj['model'])

## bandlimited trajectories